In [ ]:
from gemma import PaligemmaForConditionGeneration, PaliGemmaConfig
from transformers import AutoTokenizer
import json
import glob
from safetensors import safe_open
from typing import Tuple
import os

In [ ]:
def load_hf_model(model_path: str, device: str) -> Tuple[PaligemmaForConditionGeneration, AutoTokenizer]:
    #load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="right")
    assert tokenizer.padding_side == "right"

    #find all *.safetensors files
    safetensors_files = glob.glob(os.path.join(model_path, "*.safetensors"))

    #... and load them one by one in tensors directory
    tensors = {}
    fro safetensors_file in safetensors_files:
    with safe_open(safetensors_file, framework="pt", device="cpu") as f:
        for key in f.keys():
            tensors[key] = f.get_tensor(key)

    #load models config
    with open(os.path.join(model_path, "config.json"), "r") as f:
        model_config_file = json.load(f)
        config = PaliGemmaConfig(**model_config_file)

    #create the model using the config
    model = PaligemmaForConditionGeneration(config).to(device)

    #load the state dict of the model
    model.load_state_dict(tensors, strict=False)

    #tie weigths
    model.tie_weights()

    return (model, tokenizer)